## Pre-processing for analytic-ready lake data
This notebook preprocesses the OS NGD water features and network data to produce analytic-ready lake polygons which remove geometry gaps and segmentation, to allow intuitive data analysis (e.g. area calculation).

In [1]:
# Imports
from osdatahub import NGD, Extent
import geopandas as gpd
import os

c:\Users\MMarochov\Anaconda3\envs\osdata-man\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Inputs

# (434219.1758, 189352.6604, 468215.9871, 221677.3266) # Oxford 27700 
# (308544.2614,486751.2506,353181.3931,530156.0536) # Lake District 27700
bbox=(590740.5283,212447.9137,603982.4393,222757.4102) # Abberton Reservoir 27700
collection = "wtr-ntwk-waterlink"
country = 'eng'
union_lakes_path = f'output_data/union_lakes_{country}.gpkg'

### Get waterlink data filtered by "Still Water" and "Reservoir" description fields:

In [3]:
# Inputs to the api query
apiKey = os.environ.get("OS_API_KEY")
extent = Extent.from_bbox(bbox, crs="EPSG:27700")
ngd = NGD(apiKey, collection)

In [4]:
# filter1 = "description='Still Water'"
# filter2 = "description='Reservoir'"

`cql_filter="description='Still Water' OR description='Reservoir'"` just kept running for years so decided to do them as seperate queries and append together:

In [4]:
# Make api call
result_stillwater = ngd.query(extent=extent, max_results=9000000000, crs="EPSG:27700", cql_filter="description='Still Water'")

In [5]:
result_reservoir = ngd.query(extent=extent, max_results=9000000000, crs="EPSG:27700", cql_filter="description='Reservoir'")

Sanity checks:
- ```result_stillwater```
- ```len(result_stillwater['features'])```

In [6]:
gdf_stillwater = gpd.GeoDataFrame.from_features(result_stillwater)
gdf_reservoir = gpd.GeoDataFrame.from_features(result_reservoir)

Sanity checks:
- `len(gdf_stillwater)`
- `len(gdf_stillwater)+len(gdf_reservoir)`

In [7]:
gdf = gdf_stillwater.append(gdf_reservoir, ignore_index=True)

C:\Users\MMarochov\AppData\Local\Temp\1\ipykernel_53364\3683505347.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf_stillwater.append(gdf_reservoir, ignore_index=True)


Apply crs to geodataframe:

In [8]:
#print(gdf.crs)
gdf.set_crs(27700, inplace=True)

,geometry,osid,toid,theme,width,nameid,endnode,primacy,gradient,startnode,...,capturespecification,geometry_evidencedate,description_updatedate,nametertiary1_language,nametertiary2_language,versionavailabletodate,namesecondary1_language,namesecondary2_language,description_evidencedate,versionavailablefromdate
0,"LINESTRING (603136.044 216652.893, 603135.351 ...",007fb6eb-1668-457e-b8f2-5f9a7c5c71bd,osgb5000005193196661,Water,NaN,None,8320cf8b-9553-47ae-8a37-f3495eaf9193,1,NaN,c9348305-780b-4dec-9a96-79fd4f6d4a0e,...,None,2016-04-30,2016-10-18,None,None,None,None,None,2016-04-30,2022-09-03T00:00:00Z
1,"LINESTRING (603109.846 218234.260, 603113.914 ...",026febba-b75b-44f8-864d-eca3ec95505d,osgb5000005193196650,Water,NaN,None,5b61f18f-a1c6-4227-8fc8-6623a96df62e,1,0.22,a50f2112-7b3f-4029-996d-5fd923ee6b73,...,None,2016-04-30,2016-10-18,None,None,None,None,None,2016-04-30,2022-09-03T00:00:00Z
2,"LINESTRING (590873.440 212891.850, 590874.420 ...",0538f1e4-5523-4c4f-b26e-0fa8981b2dc9,osgb5000005148117472,Water,2.1,None,d7690796-0145-4ad0-a950-a43b5907a867,1,NaN,6f1fd8a7-dec1-4297-b410-b3a6e785f2ea,...,Rural,2013-11-26,2013-11-26,None,None,None,None,None,2013-11-26,2022-09-03T00:00:00Z
3,"LINESTRING (595069.920 221796.730, 595073.780 ...",06731669-0d03-4231-bf2a-e24ce851727c,osgb5000005193160659,Water,1.1,None,b9cefb55-79cc-4e14-aef6-ce5c9a53fcf4,1,NaN,e2580678-b3ef-4366-a310-602b5b7d5f07,...,Rural,2016-04-30,2016-11-02,None,None,None,None,None,2016-04-30,2022-09-03T00:00:00Z
4,"LINESTRING (595100.449 221751.094, 595125.027 ...",07501daf-7e1f-4201-83a9-6f9894a06729,osgb5000005149383320,Water,6.3,None,3da4468e-c74b-4df4-8f46-5e7c1435ef83,1,0.04,3ce6c97e-aaa1-4a98-9085-3bef960b060d,...,Rural,2013-11-26,2013-11-26,None,None,None,None,None,2013-11-26,2022-09-03T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,"LINESTRING (596342.232 217347.198, 596404.979 ...",edfc21bd-a924-42b8-9a41-0928912a9439,osgb5000005220863090,Water,703.2,233810d2-0112-499a-be3c-961db76dc53f,2e6fce5f-2406-4c78-9302-053e349a2b12,1,0.04,524f575a-2601-41ee-95f8-bb6a64eadaed,...,Rural,2018-01-05,2018-01-05,None,None,None,None,None,2018-01-05,2022-09-03T00:00:00Z
188,"LINESTRING (597524.408 218622.915, 597316.403 ...",f158bdd0-6b68-483e-a50d-2bca432bc82f,osgb5000005193196763,Water,NaN,None,2e6fce5f-2406-4c78-9302-053e349a2b12,1,0.00,ed0328b5-3189-4ad2-b1d1-eac3ed8d3dc1,...,None,2016-11-09,2016-11-09,None,None,None,None,None,2016-11-09,2022-09-03T00:00:00Z
189,"LINESTRING (594812.176 216672.587, 594883.357 ...",f5fb4d10-716d-4428-8312-617e4e754ec8,osgb5000005220863097,Water,73.8,233810d2-0112-499a-be3c-961db76dc53f,c8a24651-f6ca-4c54-af64-c20437b0b488,1,0.00,a2e8ba48-cb93-4f17-8522-c31ad97fceaf,...,Rural,2018-01-05,2018-01-05,None,None,None,None,None,2018-01-05,2022-09-03T00:00:00Z
190,"LINESTRING (596075.740 217127.917, 596306.127 ...",f6b18f07-eaac-4825-9221-b1c95894358a,osgb5000005148330721,Water,231.1,233810d2-0112-499a-be3c-961db76dc53f,8c78bdfd-5c12-4b8b-8f18-852d01b16682,1,NaN,37b9dc6f-7471-4516-98b1-2fbfb88ecfa8,...,Rural,2013-11-26,2013-11-26,None,None,None,None,None,2013-11-26,2022-09-03T00:00:00Z


In [15]:
# Save waterlink lines as a shapefile
# gdf.to_file("lakes_ntwrk_res_or_still.shp")

C:\Users\MMarochov\AppData\Local\Temp\1\ipykernel_51868\1982957558.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf3.to_file("lakes_ntwrk_res_or_still.shp")


In [9]:
# Remove NaNs from name1_text
gdf = gdf[gdf.name1_text.notna()]

### Spatial join intersecting lines and polygons and merge/dissolve if they share the name1_text attribute:

In [10]:
# Dissolve line geometry grouping by shared name1_text
gdf_dissolved = gdf.dissolve(by='name1_text')
gdf_dissolved = gdf_dissolved.reset_index()

Spatial join lines and polygons if they intersect (to assign lake polygons with a name attribute):

In [11]:
# Read in the lake polygons as geodataframe
union_lakes_gdf = gpd.read_file(union_lakes_path)
# Spatial join of lakes and dissolved lines that intersect and have name1_text values
union_lakes_name = union_lakes_gdf.sjoin(gdf_dissolved, how="left", predicate="intersects")
# Remove unnecessary columns
union_lakes_name = union_lakes_name[['area', 'geometry', 'name1_text']]

,area,geometry,name1_text
0,9.226800,"POLYGON ((88254.600 7676.490, 88254.550 7676.4...",NaN
1,10.710262,"POLYGON ((88012.334 8090.521, 88012.608 8090.2...",NaN
2,88.145131,"POLYGON ((87915.601 8536.516, 87917.426 8537.5...",NaN


In [12]:
# Sanity check
union_lakes_name[union_lakes_name.name1_text=='Layer Brook']

,area,geometry,name1_text
421245,2.052952e+05,"POLYGON ((594320.010 216481.572, 594324.690 21...",Layer Brook
423365,4.609388e+05,"POLYGON ((596345.870 217134.290, 596343.290 21...",Layer Brook
425445,6.147913e+06,"POLYGON ((599004.875 219712.003, 599004.412 21...",Layer Brook


Dissolve geomteries with the same name and sum area value:

In [18]:
dissolved_lakes_named = union_lakes_name.dissolve(by='name1_text', aggfunc='sum')
# Reset index
dissolved_lakes_named = dissolved_lakes_named.reset_index()
dissolved_lakes_named

,name1_text,geometry,area
0,Claypits Pond,"POLYGON ((592982.670 220140.870, 592984.370 22...",7.179500e+01
1,Layer Brook,"MULTIPOLYGON (((594324.690 216495.058, 594327....",6.814147e+06


In [19]:
# Save named lake polygons as a shapefile
dissolved_lakes_named.to_file("named_lake_polygons_AbbRes.shp")

Need to add back in the nan lakes